### Supertonic-2

In [23]:
from collections import deque

total_messages = deque(maxlen=10)
messages = [{"role": "user", "content": "안녕?"}]
messages1 = [{"role": "user", "content": "안녕?"}]
total_messages.append(messages)
total_messages.append(messages1)
total_messages

deque([[{'role': 'user', 'content': '안녕?'}],
       [{'role': 'user', 'content': '안녕?'}]],
      maxlen=10)

In [24]:
list(total_messages)

[[{'role': 'user', 'content': '안녕?'}], [{'role': 'user', 'content': '안녕?'}]]

In [24]:
import sys
sys.path.append("../supertonic/py")

from helper import load_text_to_speech, timer, sanitize_filename, load_voice_style
import soundfile as sf
from IPython.display import Audio

In [43]:
# 모델 로드
onnx_dir = "../supertonic/py/assets/supertonic-2/onnx"
text_to_speech = load_text_to_speech(onnx_dir, use_gpu=False)

# 화자 스타일 로드
style_dir = "../supertonic/assets/voice_styles/F1.json"
style = load_voice_style([style_dir], verbose=True)

# 더미 파일 생성
_ = text_to_speech("테스트", "ko", style, total_step=5, speed=1.3)

Using CPU for inference
Loaded 1 voice styles


In [45]:
text = "대전광역시청의 주소는 대전광역시 서구 내방로 111입니다."
lang = "ko"

wav, duration = text_to_speech(text, lang, style, total_step=5, speed=1.2)

output_path = './audio/supertonic_tts.wav'
sf.write(output_path, wav[0, :int(text_to_speech.sample_rate * duration[0].item())], text_to_speech.sample_rate)

Audio(output_path)

### MeloTTS

In [1]:
from memory_monitor import MemoryMonitor
import torch, psutil, os, gc, re, json
import sys
sys.path.append("/home/user/work/MeloTTS")

from melo.api import TTS
import torch
from IPython.display import Audio
import librosa
from pesq import pesq
from pystoi import stoi
import time
import whisper
from jiwer import cer

/home/user/miniconda3/envs/tts_hori/lib/python3.10/site-packages/librosa/util/files.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
/home/user/miniconda3/envs/tts_hori/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/user/miniconda3/envs/tts_hori/lib/python3.10/site-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest 

In [2]:
monitor = MemoryMonitor(device_index=0)
torch.cuda.empty_cache()
gc.collect()

monitor.print_usage(label="Before Loading")

speed = 1.2
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
output_path = './audio/melo_tts.wav'

# MeloTTS 모델
model = TTS(language='KR', device=device)
# Whisper 모델
stt_model = whisper.load_model("turbo")

speaker_ids = model.hps.data.spk2id

# 더미 파일 생성
model.tts_to_file("테스트", speaker_ids['KR'], output_path, speed=speed)

monitor.print_usage(label="After Loading")

[Before Loading] RAM: 1149.14 MB | VRAM: 0.00 MB


/home/user/miniconda3/envs/tts_hori/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


 > Text split to sentences.
테스트
 > ===========================


  0%|          | 0/1 [00:00<?, ?it/s]/home/user/miniconda3/envs/tts_hori/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/user/miniconda

[After Loading] RAM: 4599.77 MB | VRAM: 0.00 MB


In [7]:
text = "김치 3956개랑 1239를 더하면 392304개에요."

# 합성 시작 시간 기록
start_time = time.time()

model.tts_to_file(text, speaker_ids['KR'], output_path, speed=speed)

# 합성 종료 시간 기록 및 소요 시간 계산
end_time = time.time()
processing_time = end_time - start_time

# 생성된 파일의 실제 오디오 길이 측정
audio_data, sr = librosa.load(output_path, sr=None) # sr=None: 오디오 파일이 가지고 있는 원래의 샘플링 레이트 그대로 읽어옴
audio_duration = len(audio_data) / sr

# RTF 계산
rtf = processing_time / audio_duration

print(f"--- 효율성 분석 결과 ---")
print(f"문장 길이: {len(text)}자")
print(f"오디오 길이: {audio_duration:.2f}초")
print(f"합성 소요 시간: {processing_time:.2f}초")
print(f"RTF: {rtf:.4f}")

Audio(output_path)

# monitor.print_usage(label="After Loading")

 > Text split to sentences.
김치 3956개랑 1239를 더하면 392304개에요.
 > ===========================


100%|██████████| 1/1 [00:00<00:00,  1.23it/s]

--- 효율성 분석 결과 ---
문장 길이: 30자
오디오 길이: 5.61초
합성 소요 시간: 0.93초
RTF: 0.1660


In [51]:
result = stt_model.transcribe(output_path)
detected_text = result["text"].strip()

error_rate = cer(text, detected_text)

In [52]:
print(f"원래 문장: {text}")
print(f"인식 문장: {detected_text}")
print(f"CER (에러율): {error_rate:.4f} (0에 가까울수록 명확함)")

원래 문장: 서변온천의 주소는 대구광역시 북구 서변로 256길입니다.
인식 문장: 서변온천의 주소는 대구광역시 북고 서별로 2호 6길입니다.
CER (에러율): 0.1290 (0에 가까울수록 명확함)
